In [1]:
import pandas as pd
import requests, json
import time
import os
import requests as rs
from bs4 import BeautifulSoup as BS

In [2]:
print(os.listdir('./data'))
PATH_to_file = './data/'

['test.csv']


In [3]:
df_marks_and_models = pd.read_csv('all_car_brands_and_their_models_from_auto_ru.csv', index_col=0)
print('Кол-во марок: ', df_marks_and_models.shape[0])
display(df_marks_and_models.head(2))

Кол-во марок:  262


,Models
Mark,
AC,"['378_GT', 'ACE', 'ACECA', 'COBRA']"
ACURA,"['CL', 'CSX', 'EL', 'ILX', 'INTEGRA', 'LEGEND'..."


In [4]:
brands ={'AC':2, 'Acura':48, 'Alfa Romeo':40, 'Alpina':3, 
                             'AMC':2, 'Ariel':1, 'Asia':2, 'Aston Martin':13, 
                             'Audi':2702, 'Austin Healey':1, 
                             'Bentley':92, 'BMW':5978, 'Borgward':1, 
                             'Brilliance':15, 'Buick':12, 'BYD':10, 
                             'Cadillac':392, 'Changan':173, 'Chery':492, 
                             'Chevrolet':2204, 'Chrysler':205, 'Citroen':779, 
                             'Dacia':5, 'Daewoo':543, 'Daihatsu':25, 
                             'Daimler':3, 'Datsun':126, 'Delage':1, 'Derways':1, 
                             'DKW':2, 'Dodge':267, 'DongFeng':9, 'Doninvest':2, 
                             'DS':1, 'DW Hower':4, 'Eagle':2, 'Excalibur':1, 
                             'FAW':47, 'Ferrari':28, 'Fiat':170, 'Ford':2836, 
                             'Foton':1, 'GAC':15, 'Geely':289, 'Genesis':34, 
                             'GMC':18, 'Great Wall':291, 'Hafei':2, 'Haima':6, 
                             'Haval':426, 'Hawtai':3, 'Heinkel':1, 'Honda':782, 
                             'HuangHai':1, 'Hummer':49, 'Hyundai':4493, 
                             'Infiniti':619, 'Innocenti':1, 'Iran Khodro':17, 
                             'Isuzu':23, 'JAC':5, 'Jaguar':249, 'Jeep':425, 'JMC':1, 
                             'Kia':5595, 'LADA (ВАЗ)':8935, 'Lamborghini':14, 
                             'Lancia':4, 'Land Rover':1269, 'Lexus':778, 
                             'Lifan':175, 'Lincoln':49, 'Lotus':1, 'Luxgen':7, 
                             'Mahindra':1, 'Maserati':54, 'Maybach':7, 
                             'Mazda':1708, 'McLaren':2, 'Mercedes-Benz':9810, 
                             'Mercury':8, 'Metrocab':1, 'MINI':460, 
                             'Mitsubishi':3045, 'Mitsuoka':1, 'Nissan':4075, 
                             'Oldsmobile':7, 'Opel':2017, 'Packard':2, 
                             'Peugeot':1075, 'Plymouth':11, 'Pontiac':19, 
                             'Porsche':636, 'Proton':1, 'Ravon':71, 
                             'Renault':3704, 'Rolls-Royce':59, 'Rover':41, 
                             'Saab':98, 'Saturn':4, 'Scion':5, 'SEAT':63, 
                             'Shanghai Maple':1, 'Skoda':2752, 'Smart':81, 
                             'SsangYong':424, 'Subaru':573, 'Suzuki':975, 
                             'Tatra':1, 'Tesla':54, 'Tianma':1, 'Toyota':3082, 
                             'Triumph':1, 'Volkswagen':5309, 'Volvo':1137, 
                             'Vortex':49, 'Zotye':9, 'ZX':4, 'ГАЗ':712, 'Гоночный автомобиль':3, 
                             'ЗАЗ':185, 'ЗИЛ':3, 'ИЖ':85, 'ЛуАЗ':35, 
                             'Москвич':158, 'СМЗ':4, 'ТагАЗ':50, 'УАЗ':830}

In [5]:
# уберем модели с кол-вом объявлений ниже 300 и повысим регистр индексов 
list_keys_to_delete = []
for mark in brands.keys():
    if brands[mark] < 300:
        list_keys_to_delete.append(mark)
    
for key in list_keys_to_delete: del brands[key]

for mark in list(brands.keys()):
    mark_up = mark.upper()
    brands[mark_up]=brands.pop(mark)

In [6]:
brands

{'AUDI': 2702,
 'BMW': 5978,
 'CADILLAC': 392,
 'CHERY': 492,
 'CHEVROLET': 2204,
 'CITROEN': 779,
 'DAEWOO': 543,
 'FORD': 2836,
 'HAVAL': 426,
 'HONDA': 782,
 'HYUNDAI': 4493,
 'INFINITI': 619,
 'JEEP': 425,
 'KIA': 5595,
 'LADA (ВАЗ)': 8935,
 'LAND ROVER': 1269,
 'LEXUS': 778,
 'MAZDA': 1708,
 'MERCEDES-BENZ': 9810,
 'MINI': 460,
 'MITSUBISHI': 3045,
 'NISSAN': 4075,
 'OPEL': 2017,
 'PEUGEOT': 1075,
 'PORSCHE': 636,
 'RENAULT': 3704,
 'SKODA': 2752,
 'SSANGYONG': 424,
 'SUBARU': 573,
 'SUZUKI': 975,
 'TOYOTA': 3082,
 'VOLKSWAGEN': 5309,
 'VOLVO': 1137,
 'ГАЗ': 712,
 'УАЗ': 830}

In [7]:
for key in ['LADA (ВАЗ)','ГАЗ','УАЗ']: del brands[key]

In [8]:
brands_list = list(brands.keys())

In [9]:
print(brands_list)

['AUDI', 'BMW', 'CADILLAC', 'CHERY', 'CHEVROLET', 'CITROEN', 'DAEWOO', 'FORD', 'HAVAL', 'HONDA', 'HYUNDAI', 'INFINITI', 'JEEP', 'KIA', 'LAND ROVER', 'LEXUS', 'MAZDA', 'MERCEDES-BENZ', 'MINI', 'MITSUBISHI', 'NISSAN', 'OPEL', 'PEUGEOT', 'PORSCHE', 'RENAULT', 'SKODA', 'SSANGYONG', 'SUBARU', 'SUZUKI', 'TOYOTA', 'VOLKSWAGEN', 'VOLVO']


In [10]:
list_marks_df = list(df_marks_and_models.index)
list_marks_to_check = []
for mark in brands.keys():
    if mark not in list_marks_df:
        list_marks_to_check.append(mark)
print(f'всего марок с кол-вом объявлений более 300:= {len(brands)}')
print('список марок, которые нужно проверить :=', *list_marks_to_check)

всего марок с кол-вом объявлений более 300:= 32
список марок, которые нужно проверить := LAND ROVER MERCEDES-BENZ SSANGYONG


In [11]:
brands['MERCEDES']=brands.pop('MERCEDES-BENZ')
brands['SSANG_YONG']=brands.pop('SSANGYONG')
brands['LAND_ROVER']=brands.pop('LAND ROVER')

In [12]:
sum_of_ads = sum(brands.values())
print(f"примерное кол-во строк после парсинга должно быть := {sum_of_ads}")

примерное кол-во строк после парсинга должно быть := 71095


In [13]:
HEADERS = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
    'Connection': 'keep-alive',
    'Content-Length': '137',
    'content-type': 'application/json',
    'Cookie': 'autoru_gdpr=1; _csrf_token=1c0ed592ec162073ac34d79ce511f0e50d195f763abd8c24; autoru_sid=a%3Ag5e3b198b299o5jhpv6nlk0ro4daqbpf.fa3630dbc880ea80147c661111fb3270%7C1580931467355.604800.8HnYnADZ6dSuzP1gctE0Fw.cd59AHgDSjoJxSYHCHfDUoj-f2orbR5pKj6U0ddu1G4; autoruuid=g5e3b198b299o5jhpv6nlk0ro4daqbpf.fa3630dbc880ea80147c661111fb3270; suid=48a075680eac323f3f9ad5304157467a.bc50c5bde34519f174ccdba0bd791787; from_lifetime=1580933172327; from=yandex; X-Vertis-DC=myt; crookie=bp+bI7U7P7sm6q0mpUwAgWZrbzx3jePMKp8OPHqMwu9FdPseXCTs3bUqyAjp1fRRTDJ9Z5RZEdQLKToDLIpc7dWxb90=; cmtchd=MTU4MDkzMTQ3MjU0NQ==; yandexuid=1758388111580931457; bltsr=1; ',
    'Host': 'auto.ru',
    'origin': 'https://auto.ru/cars/all/',
    'Referer': 'https://auto.ru/cars/all/',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0',
    'x-client-app-version': '202002.03.092255',
    'x-client-date': '1580933207763',
    'x-csrf-token': '1c0ed592ec162073ac34d79ce511f0e50d195f763abd8c24',
    'x-page-request-id': '60142cd4f0c0edf51f96fd0134c6f02a',
    'x-requested-with': 'fetch'
}

In [14]:
url = "https://auto.ru/-/ajax/desktop/listing/"

In [15]:
test = pd.read_csv(PATH_to_file+'test.csv')
print('Кол-во столбцов теста:', test.shape[1])
print('Названия столбцов теста:', list(test.columns))

pd.set_option('display.max_columns', None)
test.head(2)

Кол-во столбцов теста: 32
Названия столбцов теста: ['bodyType', 'brand', 'car_url', 'color', 'complectation_dict', 'description', 'engineDisplacement', 'enginePower', 'equipment_dict', 'fuelType', 'image', 'mileage', 'modelDate', 'model_info', 'model_name', 'name', 'numberOfDoors', 'parsing_unixtime', 'priceCurrency', 'productionDate', 'sell_id', 'super_gen', 'vehicleConfiguration', 'vehicleTransmission', 'vendor', 'Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль', 'Состояние', 'Таможня']


,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,синий,NaN,"Все автомобили, представленные в продаже, прох...",1.2 LTR,105 N12,"{""engine-proof"":true,""tinted-glass"":true,""airb...",бензин,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,74000,2013,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1.2 AMT (105 л.с.),5,1603226273,RUB,2014,1100575026,"{""id"":""10373605"",""displacement"":1197,""engine_t...",LIFTBACK ROBOT 1.2,роботизированная,EUROPEAN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,чёрный,NaN,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6 LTR,110 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,60563,2017,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1.6 MT (110 л.с.),5,1603226277,RUB,2017,1100549428,"{""id"":""20913311"",""displacement"":1598,""engine_t...",LIFTBACK MECHANICAL 1.6,механическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен


In [16]:
test.equipment_dict[1]

'{"cruise-control":true,"asr":true,"esp":true,"airbag-driver":true,"isofix":true,"usb":true,"light-sensor":true,"airbag-passenger":true,"computer":true,"wheel-power":true,"alarm":true,"lock":true,"fabric-seats":true,"electro-mirrors":true,"drl":true,"electro-window-back":true,"alloy-wheel-disks":true,"16-inch-wheels":true,"ptf":true,"rain-sensor":true,"airbag-side":true,"tyre-pressure":true,"mirrors-heat":true,"dark-interior":true,"driver-seat-updown":true,"wheel-heat":true,"abs":true,"front-seats-heat":true,"bluetooth":true,"wheel-configuration2":true,"immo":true,"windscreen-heat":true,"climate-control-2":true,"park-assist-r":true}'

In [17]:
# сделаем тестовый вызов запроса для BMW модели M6 и выведем его на экран, чтобы по нему можно было сделать функцию для создания 23 необходимых столбцов
# но сначала установим главные параметры запроса
mark = 'BMW'  # марка
model = 'M6'  # модель
geo_radius = 200  # радиус вокруг москвы = 200 км
geo_id = [213]  # Москва
top_days = "1"  # кол-во дней от текущей даты назад за которые запрашиваем объявления 

param = {
                'catalog_filter' : [{"mark": mark, "model": model}],
                'section': "all",
                'category': "cars",
                'sort': "fresh_relevance_1-desc",
                "geo_radius":geo_radius,
                "geo_id":geo_id,
                'page': 1
            }

response = requests.post(url, json=param, headers = HEADERS)
data = response.json()
for elem in data['offers']:
    print(elem)
    break

{'availability': 'IN_STOCK', 'category': 'cars', 'color_hex': '0000CC', 'delivery_info': {}, 'description': 'Премиальная акустическая система объемного звука Bang & Olufsen с 16 динамиками\nАдаптивный круиз-контроль\nСигнализация схода с полосы движения\nПроекция на лобовое стекло\n\nАвтомобиль в идеальном внешнем и техническом состоянии.\nОбслуживание автомобиля исключительно у официального дилера, все ТО согласно регламенту.\nСЕМЬ регистрационных действий, ТРИ Владельца за все время эксплуатации.\n\n"Royal Motors Club"  предоставляет полный спектр услуг по продажам автомобилей с пробегом:\n- Выгодные условия кредитования, одобрение по двум документам.\n- Trade In – обмен любого Вашего авто на представленный в нашей компании это\n   быстро, выгодно, безопасно!\n-  Мы осуществили все необходимые проверки и даем гарантию юридической\n   чистоты всех наших автомобилей.\n - Оптимальные программы по авто-страхованию ( ОСАГО, КАСКО)\n\nАвтомобиль прошел комплексную диагностику и предпродажн

In [18]:
elem['additional_info']['hidden']

False

In [19]:
def utils_clean_values_dict(d_dict):
    for elem in d_dict:
        d_dict[elem] = 0
    return

In [20]:
# создаем вспомагательный словарь, в который будем записывать необходимые данные из каждого объявления
dict_columns = {'bodyType':0, 'brand':0, 'car_url':0, 'color':0, 'complectation_dict':0,
       'description':0, 'engineDisplacement':0, 'enginePower':0, 'equipment_dict':0,
       'fuelType':0, 'image':0, 'mileage':0, 'modelDate':0, 'model_info':0, 'model_name':0,
       'name':0, 'numberOfDoors':0, 'parsing_unixtime':0, 'priceCurrency':0,
       'productionDate':0, 'sell_id':0, 'super_gen':0, 'vehicleConfiguration':0,
       'vehicleTransmission':0, 'vendor':0, 'Владельцы':0, 'Владение':0, 'ПТС':0,
       'Привод':0, 'Руль':0, 'Состояние':0, 'Таможня':0, 'price':0}


# создаем функцию для обработки 23 необходимых столбцов из объявления в словарь
def from_elem_to_dict(d_elem, d_dict):
    utils_clean_values_dict(d_dict)
    
    try: d_dict['bodyType'] = d_elem['vehicle_info']['configuration']['human_name']
    except: d_dict['bodyType'] = None

    try: d_dict['brand'] = d_elem['vehicle_info']['mark_info']['name']
    except: d_dict['brand'] = None

    d_dict['car_url'] = None
        
    try: d_dict['color'] = d_elem['color_hex']
    except: d_dict['color'] = None
        
    try: d_dict['complectation_dict'] = d_elem['vehicle_info']['complectation']
    except: d_dict['complectation_dict'] = None
   
    try: d_dict['description'] = d_elem['description']
    except: d_dict['description'] = None    
        
    try: d_dict['engineDisplacement'] = d_elem['vehicle_info']['tech_param']['displacement']
    except: d_dict['engineDisplacement'] = None
    
    try: d_dict['enginePower'] = d_elem['vehicle_info']['tech_param']['power']
    except: d_dict['enginePower'] = None
    
    try: d_dict['equipment_dict'] = elem['vehicle_info']['equipment']
    except: d_dict['equipment_dict'] = None

    try: d_dict['fuelType'] = d_elem['vehicle_info']['tech_param']['engine_type']
    except: d_dict['fuelType'] = None    
        
    d_dict['image'] = None
        
        
    try: d_dict['mileage'] = d_elem['state']['mileage']
    except: d_dict['mileage'] = None       
    
    try: d_dict['modelDate'] = d_elem['vehicle_info']['super_gen']['year_from']
    except: d_dict['modelDate'] = None
            
    try: d_dict['model_info'] = d_elem['vehicle_info']['model_info']
    except: d_dict['model_info'] = None

    try: d_dict['model_name'] =  d_elem['vehicle_info']['model_info']['code']
    except: d_dict['model_name'] = None
        
    try: d_dict['name'] =  d_elem['vehicle_info']['tech_param']['human_name']
    except: d_dict['name'] = None
    
    try: d_dict['numberOfDoors'] = d_elem['vehicle_info']['configuration']['doors_count']
    except: d_dict['numberOfDoors'] = None
        
    try: d_dict['parsing_unixtime'] = d_elem['additional_info']['fresh_date']
    except: d_dict['parsing_unixtime'] = None

    try: d_dict['priceCurrency'] = d_elem['price_info']['currency']
    except: d_dict['priceCurrency'] = None
    
    try: d_dict['productionDate'] = d_elem['documents']['year']
    except: d_dict['productionDate'] = None

    
    try: d_dict['sell_id'] = d_elem['price_history']['saleid']
    except: d_dict['sell_id'] = None
        
    try: d_dict['super_gen'] = d_elem['vehicle_info']['super_gen'] 
    except: d_dict['super_gen'] = None
    
    try: d_dict['vehicleConfiguration'] = elem['vehicle_info']['configuration']
    except: d_dict['vehicleConfiguration'] = None
    
    try: d_dict['vehicleTransmission'] = d_elem['vehicle_info']['tech_param']['transmission']
    except: d_dict['vehicleTransmission'] = None
        
    try: d_dict['vendor'] = d_elem['vehicle_info']['vendor']
    except: d_dict['vendor'] = None

    
    try: d_dict['Владельцы'] = d_elem['documents']['owners_number']
    except: d_dict['Владельцы'] = None

    try: d_dict['Владение'] =  d_elem['documents']['purchase_date']
    except: d_dict['Владение'] = None

    try: d_dict['ПТС'] = d_elem['documents']['pts']
    except: d_dict['ПТС'] = None

    try: d_dict['Привод'] = d_elem['vehicle_info']['tech_param']['gear_type']
    except: d_dict['Привод'] = None

    try: d_dict['Руль'] = d_elem['vehicle_info']['steering_wheel']
    except: d_dict['Руль'] = None
    
     # 'Состояние' не найдено
    d_dict['Состояние'] = None

    try: d_dict['Таможня'] = d_elem['documents']['custom_cleared']
    except: d_dict['Таможня'] = None

    try: d_dict['price'] = d_elem['price_info']['RUR']
    except: d_dict['price'] = None

    return

In [21]:
# тест функции 
from_elem_to_dict(elem, dict_columns)

In [22]:
# проверяем формат тестового запроса с сайта и теста с кагл
display(pd.DataFrame(dict_columns).sample(2))
test[test['enginePower']=='560 N12'].sample(2)

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,price
light-cleaner,Купе,BMW,None,0000CC,NaN,Премиальная акустическая система объемного зву...,4395,560,True,GASOLINE,None,67000,2012,NaN,M6,4.4 AMT (560 л.с.),2,1609665239000,RUR,2012,None,NaN,NaN,ROBOT,EUROPEAN,3,None,DUPLICATE,REAR_DRIVE,LEFT,None,True,2750000
available_options,Купе,BMW,None,0000CC,"[cruise-control, sport-seats, multi-wheel, xen...",Премиальная акустическая система объемного зву...,4395,560,NaN,GASOLINE,None,67000,2012,NaN,M6,4.4 AMT (560 л.с.),2,1609665239000,RUR,2012,None,NaN,NaN,ROBOT,EUROPEAN,3,None,DUPLICATE,REAR_DRIVE,LEFT,None,True,2750000


,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
30080,универсал 5 дв.,AUDI,https://auto.ru/cars/used/sale/audi/rs6/110130...,синий,NaN,Эксплуатация с декабря 2015 года. \nМодель 201...,4.0 LTR,560 N12,"{""esp"":true,""adaptive-light"":true,""multi-wheel...",бензин,https://avatars.mds.yandex.net/get-autoru-vos/...,107000,2012,"{""code"":""RS6"",""name"":""RS 6"",""ru_name"":""РС6"",""m...",RS6,4.0 AT (560 л.с.) 4WD,5,1603567840,RUB,2013,1101307562,"{""id"":""9360687"",""displacement"":3993,""engine_ty...",WAGON_5_DOORS AUTOMATIC 4.0,автоматическая,EUROPEAN,2 владельца,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен
8072,седан,BMW,https://auto.ru/cars/used/sale/bmw/m5/11003155...,белый,"{""id"":""7742579"",""name"":""M5"",""available_options...","Автомобиль в хорошем состоянии, своевременное ...",4.4 LTR,560 N12,"{""cruise-control"":true,""esp"":true,""sport-seats...",бензин,https://avatars.mds.yandex.net/get-autoru-vos/...,68000,2011,"{""code"":""M5"",""name"":""M5"",""ru_name"":""М5"",""morph...",M5,4.4 AMT (560 л.с.),4,1603109714,RUB,2012,1100315592,"{""id"":""7742577"",""displacement"":4395,""engine_ty...",SEDAN ROBOT 4.4,роботизированная,EUROPEAN,3 или более,NaN,Дубликат,задний,Левый,Не требует ремонта,Растаможен


In [23]:
from time import perf_counter 
import time

In [24]:
auto_ru = {}

iter_of_ads, iter_of_marks, old_iter_of_ads, sum_ads, sum_ads_from_dict = 0,0,0,0,0
space = ' '
start_time_global = perf_counter()

for mark in brands_list:
    
    list_of_models = json.loads(df_marks_and_models.loc[mark, 'Models'].replace("'", '"'))
    for model in list_of_models:
        start_time = perf_counter()
        for num_page in range(1, 2):
            param = {
                'catalog_filter' : [{"mark": mark, "model": model}],
                'section': "all",
                'category': "cars",
                'sort': "fresh_relevance_1-desc",
                "top_days":"900",
                "geo_radius":400,
                "geo_id":[213],
                'page': num_page
            }
            response = requests.post(url, json=param, headers = HEADERS)
            data = response.json()

            if data['offers'] != []:
                for elem in data['offers']:
                    from_elem_to_dict(elem, dict_columns)
                    auto_ru[iter_of_ads] = dict_columns.copy()
                    iter_of_ads += 1
            else:
                break
    iter_of_marks +=1
    temp_1 = iter_of_ads-old_iter_of_ads
    temp_2 = brands[mark]
    temp_3 = temp_1/temp_2
    temp_4 = space*abs(13-len(mark)-len(str(iter_of_marks)))
    temp_5 = int(time.perf_counter() - start_time)
    temp_8 = round(temp_3*100,2)
    temp_7 = space*abs(18-len(str(temp_1))-len(str(temp_2))-len(str(temp_8))-len(str(temp_5)))
    temp_9 = int(perf_counter() - start_time_global)
    

    sum_ads += temp_1
    sum_ads_from_dict += temp_2
    temp_def = sum_ads/sum_ads_from_dict
    temp_6 = (sum_of_ads*temp_def - iter_of_ads)*temp_9/sum_ads


    print (f'{iter_of_marks}.{mark} {temp_4} ушло:{temp_5} сек., стр. {temp_1} из {temp_2} {temp_8}%,{temp_7}всего стр. {iter_of_ads}, прогноз {int(temp_6/60)} мин.')

    old_iter_of_ads=iter_of_ads

print (f'Всего спарсили := {iter_of_ads} строк')

1.AUDI          ушло:0 сек., стр. 743 из 2702 27.5%,      всего стр. 743, прогноз 97 мин.
2.BMW           ушло:1 сек., стр. 767 из 5978 12.83%,     всего стр. 1510, прогноз 40 мин.
3.CADILLAC      ушло:0 сек., стр. 153 из 392 39.03%,      всего стр. 1663, прогноз 42 мин.
4.CHERY         ушло:0 сек., стр. 361 из 492 73.37%,      всего стр. 2024, прогноз 42 мин.
5.CHEVROLET     ушло:0 сек., стр. 687 из 2204 31.17%,     всего стр. 2711, прогноз 42 мин.
6.CITROEN       ушло:0 сек., стр. 462 из 779 59.31%,      всего стр. 3173, прогноз 43 мин.
7.DAEWOO        ушло:0 сек., стр. 192 из 543 35.36%,      всего стр. 3365, прогноз 43 мин.
8.FORD          ушло:0 сек., стр. 763 из 2836 26.9%,      всего стр. 4128, прогноз 41 мин.
9.HAVAL         ушло:3 сек., стр. 139 из 426 32.63%,      всего стр. 4267, прогноз 40 мин.
10.HONDA        ушло:0 сек., стр. 422 из 782 53.96%,      всего стр. 4689, прогноз 44 мин.
11.HYUNDAI      ушло:0 сек., стр. 830 из 4493 18.47%,     всего стр. 5519, прогноз 36 мин.


KeyError: 'LAND ROVER'

In [65]:
df_auto_ru = pd.DataFrame(auto_ru).T
df_auto_ru.to_csv('auto_ru_24_01_2021.csv', index=False)